Bovard's [random agent](https://www.kaggle.com/code/bovard/kore-python-random-agent) is implemented using `env.run` interface. This notebook shows how to control each player in each step.

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
print(env.name, env.version)

In [ ]:
from kaggle_environments.envs.kore_fleets.helpers import *


state_dicts = env.reset(num_agents=2)
while not env.done:
    all_actions = []
    
    common_obs = {common_key: state_dicts[0]['observation'][common_key] for common_key in ('players', 'step', 'kore')}
    for player_state in state_dicts:
        player_state['observation'].update(common_obs)
        board = Board(player_state['observation'], env.configuration)
        
        me = board.current_player
        turn = board.step
        spawn_cost = board.configuration.spawn_cost
        kore_left = me.kore
        
        for shipyard in me.shipyards:
            if shipyard.ship_count > 10:
                direction = Direction.from_index(turn % 4)
                action = ShipyardAction.launch_fleet_with_flight_plan(2, direction.to_char())
                shipyard.next_action = action
            elif kore_left > spawn_cost * shipyard.max_spawn:
                action = ShipyardAction.spawn_ships(shipyard.max_spawn)
                shipyard.next_action = action
                kore_left -= spawn_cost * shipyard.max_spawn
            elif kore_left > spawn_cost:
                action = ShipyardAction.spawn_ships(1)
                shipyard.next_action = action
                kore_left -= spawn_cost
        all_actions.append(me.next_actions)
    state_dicts = env.step(all_actions)

In [ ]:
print('Rewards:')
for player_state in state_dicts:
    print(f"Player {player_state['observation']['player']}: {player_state['reward']:.2f}")

In [ ]:
env.render(mode="ipython", width=1000, height=800)

The same code is valid for 4-player game

In [ ]:
state_dicts = env.reset(num_agents=4)
while not env.done:
    all_actions = []
    
    common_obs = {common_key: state_dicts[0]['observation'][common_key] for common_key in ('players', 'step', 'kore')}
    for player_state in state_dicts:
        player_state['observation'].update(common_obs)
        board = Board(player_state['observation'], env.configuration)
        
        me = board.current_player
        turn = board.step
        spawn_cost = board.configuration.spawn_cost
        kore_left = me.kore
        
        for shipyard in me.shipyards:
            if shipyard.ship_count > 10:
                direction = Direction.from_index(turn % 4)
                action = ShipyardAction.launch_fleet_with_flight_plan(2, direction.to_char())
                shipyard.next_action = action
            elif kore_left > spawn_cost * shipyard.max_spawn:
                action = ShipyardAction.spawn_ships(shipyard.max_spawn)
                shipyard.next_action = action
                kore_left -= spawn_cost * shipyard.max_spawn
            elif kore_left > spawn_cost:
                action = ShipyardAction.spawn_ships(1)
                shipyard.next_action = action
                kore_left -= spawn_cost
        all_actions.append(me.next_actions)
    state_dicts = env.step(all_actions)

In [ ]:
env.render(mode="ipython", width=1000, height=800)